## Setup

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP
import scipy.io
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import LogisticRegression
from pyrcn.echo_state_network import ESNClassifier
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import cupy as cp
import reservoirpy as rpy
from reservoirpy.nodes import Reservoir
from reservoirpy.nodes import Ridge
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import itertools
import plotly.express as px
from sklearn.decomposition import PCA
import numpy as np
from reservoirpy.model import Model
from sklearn.metrics import accuracy_score


## Parameters

In [2]:
subject_id = 1

## Load data

In [3]:
npz_filename_T = f'./data/train/A0{subject_id}T.npz'
npz_filename_E = f'./data/eval/A0{subject_id}E.npz'

data_T = np.load(npz_filename_T)
data_E = np.load(npz_filename_E)

In [4]:
X_train_left = data_T['X_left']
X_train_right = data_T['X_right']
X_train_central = data_T['X_central']

# y_train = np.asarray([np.asarray([item-6 for i in range(X_train.shape[1])]) for item in data_T['y']])
y_train = data_T['y'] - 6

X_eval_left = data_E['X_left']
X_eval_right = data_E['X_right']
X_eval_central = data_E['X_central']
y_true = data_E['y_true']

In [5]:
X_train_left.shape, X_train_right.shape, X_train_central.shape

((288, 8, 751), (288, 8, 751), (288, 6, 751))

In [6]:
X_eval_left.shape, X_eval_right.shape, X_eval_central.shape, y_true.shape

((288, 8, 751), (288, 8, 751), (288, 6, 751), (288, 1))

## One-Hot encode labels

In [7]:
lb = LabelBinarizer()

y_train_onehot = lb.fit_transform(y_train)
y_true_onehot = lb.transform(y_true)

## CSP Transformation

In [8]:
# LEFT TRANSFORMATION
csp_left = CSP(n_components=4, reg=None, log=True, norm_trace=False)
csp_left = csp_left.fit(X_train_left, y_train)

X_train_left_csp = np.asarray([csp_left.transform(sample.T) for sample in X_train_left])
X_eval_left_csp = np.asarray([csp_left.transform(sample.T) for sample in X_eval_left])

# RIGHT TRANSFORMATION
csp_right = CSP(n_components=4, reg=None, log=True, norm_trace=False)
csp_right = csp_right.fit(X_train_right, y_train)

X_train_right_csp = np.asarray([csp_right.transform(sample.T) for sample in X_train_right])
X_eval_right_csp = np.asarray([csp_right.transform(sample.T) for sample in X_eval_right])

# CENTRAL TRANSFORMATION
csp_central = CSP(n_components=4, reg=None, log=True, norm_trace=False)
csp_central = csp_central.fit(X_train_central, y_train)

X_train_central_csp = np.asarray([csp_central.transform(sample.T) for sample in X_train_central])
X_eval_central_csp = np.asarray([csp_central.transform(sample.T) for sample in X_eval_central])

Computing rank from data with rank=None
    Using tolerance 2.2e-05 (2.2e-16 eps * 8 dim * 1.2e+10  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.3e-05 (2.2e-16 eps * 8 dim * 1.3e+10  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.5e-05 (2.2e-16 eps * 6 dim * 1.1

In [9]:
X_train_left_csp.shape, X_train_right_csp.shape, X_train_central_csp.shape

((288, 751, 4), (288, 751, 4), (288, 751, 4))

In [10]:
X_eval_left_csp.shape, X_eval_right_csp.shape, X_eval_central_csp.shape

((288, 751, 4), (288, 751, 4), (288, 751, 4))

## Train

In [ ]:
def extract_states(X_data, reservoir, aggregate='mean'):
    states_list = []

    for i in range(len(X_data)):
        data = X_data[i]

        states = reservoir.run(data, workers=-1)

        if aggregate == 'final':
            feature = states[-1:, :]
        elif aggregate == 'mean':
            feature = np.mean(states, axis=0, keepdims=True)
        elif aggregate == 'concat':
            # Combine multiple statistics
            feature = np.concatenate([
                states[-1:, :],
                np.mean(states, axis=0, keepdims=True),
                np.std(states, axis=0, keepdims=True)
            ], axis=1)
        
        states_list.append(feature)
    
    return np.vstack(states_list)

def extract_states_partial(X_data, reservoir, aggregate='mean'):
    states_list = []

    # print(X_data.shape)
    for i in range(len(X_data)):
        data = X_data[i]

        states = reservoir.run(data, workers=-1)

        # print(data.shape, states.shape)
        # break

        # if aggregate == 'final':
        #     feature = states[-1:, :]
        # elif aggregate == 'mean':
        #     feature = np.mean(states, axis=0, keepdims=True)
        # elif aggregate == 'concat':
        #     # Combine multiple statistics
        #     feature = np.concatenate([
        #         states[-1:, :],
        #         np.mean(states, axis=0, keepdims=True),
        #         np.std(states, axis=0, keepdims=True)
        #     ], axis=1)
        
        states_list.append(states)
    
    return np.asarray(states_list)
    return np.vstack(states_list)

In [ ]:
X_train_left_csp_states = None
X_eval_left_csp_states = None

X_train_right_csp_states = None
X_eval_right_csp_states = None

X_train_central_csp_states = None
X_eval_central_csp_states = None

In [ ]:
rpy.set_seed(42)

rs = 500
lr = 0.9
sr = 2.0

# Expected format: (time_steps, channels)
reservoir_left = Reservoir(rs, lr=lr, sr=sr)
reservoir_right = Reservoir(rs, lr=lr, sr=sr)
reservoir_central = Reservoir(rs, lr=lr, sr=sr)
reservoir_integrate = Reservoir(rs, lr=lr, sr=sr)

aggregate = 'mean'

X_train_left_csp_states = extract_states_partial(X_train_left_csp, reservoir_left, aggregate=aggregate)
X_eval_left_csp_states = extract_states_partial(X_eval_left_csp, reservoir_left, aggregate=aggregate)

X_train_right_csp_states = extract_states_partial(X_train_right_csp, reservoir_right, aggregate=aggregate)
X_eval_right_csp_states = extract_states_partial(X_eval_right_csp, reservoir_right, aggregate=aggregate)

X_train_central_csp_states = extract_states_partial(X_train_central_csp, reservoir_central, aggregate=aggregate)
X_eval_central_csp_states = extract_states_partial(X_eval_central_csp, reservoir_central, aggregate=aggregate)

In [25]:
X_train_left_csp_states.shape, X_train_right_csp_states.shape, X_train_central_csp_states.shape

((288, 751, 500), (288, 751, 500), (288, 751, 500))

In [26]:
X_train_integrate = np.concat((X_train_left_csp_states, X_train_right_csp_states, X_train_central_csp_states), axis=2)
X_eval_integrate = np.concat((X_eval_left_csp_states, X_eval_right_csp_states, X_eval_central_csp_states), axis=2)

# X_train_integrate = X_train_integrate.reshape(X_train_integrate.shape[0], 1, X_train_integrate.shape[1])
# X_eval_integrate = X_eval_integrate.reshape(X_eval_integrate.shape[0], 1, X_eval_integrate.shape[1])

X_train_integrate.shape, X_eval_integrate.shape, y_train_onehot.shape

((288, 751, 1500), (288, 751, 1500), (288, 4))

In [27]:
integrated_train_states = extract_states(X_train_integrate, reservoir_integrate, aggregate=aggregate)
integrated_eval_states = extract_states(X_eval_integrate, reservoir_integrate, aggregate=aggregate)

In [19]:
integrated_train_states.shape, integrated_eval_states.shape

((288, 500), (288, 500))

In [28]:
readout = Ridge(ridge=1e1)

readout.fit(integrated_train_states, y_train_onehot)

Ridge(ridge:10.0, input_dim:500, output_dim:4)

In [29]:
y_train_csp_pred = readout.run(integrated_train_states)
y_eval_csp_pred = readout.run(integrated_eval_states)

y_train_pred_class = np.argmax(y_train_csp_pred, axis=1) + 1
y_eval_pred_class = np.argmax(y_eval_csp_pred, axis=1) + 1

print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred_class):.4f}")
print(f'Train Kappa: {cohen_kappa_score(y_train, y_train_pred_class)}')
print(f"Eval Accuracy: {accuracy_score(y_true, y_eval_pred_class):.4f}")
print(f'Eval Kappa: {cohen_kappa_score(y_true, y_eval_pred_class)}')

print("\nClassification Report:")
print(classification_report(y_true, y_eval_pred_class))

Train Accuracy: 1.0000
Train Kappa: 1.0
Eval Accuracy: 0.2917
Eval Kappa: 0.05555555555555558

Classification Report:
              precision    recall  f1-score   support

           1       0.36      0.39      0.37        72
           2       0.23      0.21      0.22        72
           3       0.31      0.36      0.33        72
           4       0.24      0.21      0.22        72

    accuracy                           0.29       288
   macro avg       0.29      0.29      0.29       288
weighted avg       0.29      0.29      0.29       288



In [ ]:
np.random.seed(42)

# 1. Your original PCA (unchanged)
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_train_csp_states)

print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {np.sum(pca.explained_variance_ratio_):.3f}")

# 2. Define labels and colors (unchanged)
label_names = {1: 'Left Hand', 2: 'Right Hand', 3: 'Feet', 4: 'Tongue'}
colors = {1: 'red', 2: 'blue', 3: 'green', 4: 'orange'}

# 3. Prepare data for Plotly
# Create a DataFrame from your PCA results
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca['label'] = y_train

# Map numeric labels to meaningful string names (better for legends)
df_pca['Class'] = df_pca['label'].map(label_names)

# 4. Create the color map for Plotly
# This maps the *string name* (e.g., 'Left Hand') to the *color* (e.g., 'red')
color_discrete_map = {label_names[key]: colors[key] for key in label_names}

# 5. Create the interactive 3D plot
fig = px.scatter_3d(
    df_pca,
    x='PC1',
    y='PC2',
    z='PC3',
    color='Class',  # Column to use for coloring
    color_discrete_map=color_discrete_map, # Apply your specific colors
    title='Interactive 3D PCA of Reservoir States',
    
    # Add the variance info to the axis labels
    labels={
        'PC1': f'PC1 ({pca.explained_variance_ratio_[0]:.2%})',
        'PC2': f'PC2 ({pca.explained_variance_ratio_[1]:.2%})',
        'PC3': f'PC3 ({pca.explained_variance_ratio_[2]:.2%})'
    },
    
    # Customize what you see on hover
    hover_data={'Class': True, 'label': False, 'PC1': ':.3f', 'PC2': ':.3f', 'PC3': ':.3f'}
)

# Optional: Make markers smaller to match your 's=50' (Plotly's default is a bit larger)
fig.update_traces(marker=dict(size=4, opacity=0.7))

# 6. Show the figure
# In a VSCode Notebook, this will render the fully interactive plot
fig.show()

In [ ]:
X_train_csp_states.shape, y_train_onehot.shape

In [ ]:
all_trial_predictions = []

washout_time = 125

sample_step = 10

# Loop over each trial
for trial_data in X_eval_csp:
    states = []
    trial_predictions = []

    for i in range(0, trial_data.shape[0], sample_step):
        input = trial_data[i].reshape(1, trial_data[i].shape[0])
        
        state = reservoir.run(input)

        if i >= washout_time:
            states.append(state)

            mean_state = np.mean(states, axis=0, keepdims=True)

            y_pred = readout.run(mean_state).flatten()
            y_pred_class = np.argmax(y_pred) + 1


            trial_predictions.append(y_pred_class)

    all_trial_predictions.append(trial_predictions)

In [ ]:
np.array(all_trial_predictions).T.shape

In [ ]:
# Transpose the prediction matrix so we can score each time point
predictions_by_time = np.array(all_trial_predictions).T

kappa_scores = []
for y_pred_at_time_t in predictions_by_time:
    y_pred_mapped = np.array([pred for pred in y_pred_at_time_t])
    
    kappa = cohen_kappa_score(y_true, y_pred_mapped)
    kappa_scores.append(kappa)
    acc = accuracy_score(y_true, y_pred_mapped)

# --- 5. Find the Final Score ---
max_kappa = np.max(kappa_scores)
max_acc = np.max(acc)

print(f"Time course of Kappa calculated.")
print(f"Maximum Kappa value: {max_kappa:.3f}")
print(f"Maximum Accuracy value: {max_acc:.3f}")

In [ ]:
plt.plot(kappa_scores)

## Fine Tune

In [ ]:
def build_model(rs, lr, sr, alpha):
    reservoir = Reservoir(rs, lr=lr, sr=sr)
    readout = Ridge(ridge=alpha)

    return reservoir, readout

In [ ]:
rs_ls = [500, 1000, 1500]
alpha_ls = [1e0, 1e2, 1e4]
lr_ls = [0.3, 0.6, 0.9]
sr_ls = [0.7, 0.9, 1.2, 1.5, 2.0]
aggregate_ls = ['final', 'mean', 'concat']

rpy.set_seed(42)

results_list = []

model_params = list(itertools.product(rs_ls, lr_ls, sr_ls, alpha_ls, aggregate_ls))
total_iterations = len(model_params)

pbar = tqdm(total=total_iterations, desc="Grid Search Progress")

max_kappa = -1

for (rs, lr, sr, alpha, aggregate) in model_params:
    params = {'max_kappa': max_kappa, 'rs': rs, 'lr': lr, 'sr': sr, 'alpha': alpha}
    pbar.set_postfix(params)

    reservoir, readout = build_model(rs=rs, lr=lr, sr=sr, alpha=alpha)

    X_train_csp_states = extract_states(X_train_csp, reservoir, aggregate=aggregate)
    X_eval_csp_states = extract_states(X_eval_csp, reservoir, aggregate=aggregate)

    readout.fit(X_train_csp_states, y_train_onehot)

    y_train_csp_pred = readout.run(X_train_csp_states)
    y_eval_csp_pred = readout.run(X_eval_csp_states)

    y_train_pred_class = np.argmax(y_train_csp_pred, axis=1) + 1
    y_eval_pred_class = np.argmax(y_eval_csp_pred, axis=1) + 1

    acc_train = accuracy_score(y_train, y_train_pred_class)
    kappa_train = cohen_kappa_score(y_train, y_train_pred_class)

    acc_eval = accuracy_score(y_true, y_eval_pred_class)
    kappa_eval = cohen_kappa_score(y_true, y_eval_pred_class)

    f1_score_eval = f1_score(y_true, y_eval_pred_class, average='weighted')

    current_result = {
        'rs': rs,
        'lr': lr,
        'sr': sr,
        'alpha': alpha,
        'aggregate': aggregate,
        'acc_train': acc_train,
        'kappa_train': kappa_train,
        'acc_eval': acc_eval,
        'kappa_eval': kappa_eval,
        'f1_weighted': f1_score_eval
    }

    
    if kappa_eval > max_kappa:
        max_kappa = kappa_eval

    results_list.append(current_result)

    pbar.update(1)

print("Parameter search complete.")
                    
df_results = pd.DataFrame(results_list)

df_results.to_csv('tuning_results.csv', index=False)

In [ ]:
df_sorted = df_results.sort_values(by='kappa_eval', ascending=False)

print("Top 5 Results:")
print(df_sorted.head(5))

best_params = df_sorted.iloc[0].to_dict()

print("\nBest Parameter Set:")
print(best_params)

## Per subject

In [ ]:
subject_id_ls = [i for i in range(1, 10)]

for subject_id in tqdm(subject_id_ls):
    # Load data
    npz_filename_T = f'./data/train/A0{subject_id}T.npz'
    npz_filename_E = f'./data/eval/A0{subject_id}E.npz'

    data_T = np.load(npz_filename_T)
    data_E = np.load(npz_filename_E)

    # 
    X_train = data_T['X']

    # y_train = np.asarray([np.asarray([item-6 for i in range(X_train.shape[1])]) for item in data_T['y']])
    y_train = data_T['y'] - 6

    X_eval = data_E['X']
    y_true = data_E['y_true']

    # 
    X_train = data_T['X']

    y_train = data_T['y'] - 6

    X_eval = data_E['X']
    y_true = data_E['y_true']

    lb = LabelBinarizer()

    y_train_onehot = lb.fit_transform(y_train)
    y_true_onehot = lb.transform(y_true)

    csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

    csp = csp.fit(X_train, y_train)

    X_train_csp = np.asarray([csp.transform(sample.T) for sample in X_train])
    X_eval_csp = np.asarray([csp.transform(sample.T) for sample in X_eval])

    aggregate = 'mean'

    X_train_csp_states = extract_states(X_train_csp, reservoir, aggregate=aggregate)
    X_eval_csp_states = extract_states(X_eval_csp, reservoir, aggregate=aggregate)

    readout.fit(X_train_csp_states, y_train_onehot)

    y_train_csp_pred = readout.run(X_train_csp_states)
    y_eval_csp_pred = readout.run(X_eval_csp_states)

    y_train_pred_class = np.argmax(y_train_csp_pred, axis=1) + 1
    y_eval_pred_class = np.argmax(y_eval_csp_pred, axis=1) + 1

    print(25*'==')
    print(f'SUBJECT ID: {subject_id}')
    print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred_class):.4f}")
    print(f'Train Kappa: {cohen_kappa_score(y_train, y_train_pred_class)}')
    print(f"Eval Accuracy: {accuracy_score(y_true, y_eval_pred_class):.4f}")
    print(f'Eval Kappa: {cohen_kappa_score(y_true, y_eval_pred_class)}')

    print("\nClassification Report:")
    print(classification_report(y_true, y_eval_pred_class))
    
